# 3 -  Internal Validation with Boostrap

This Jupyter notebook contains the code to load the models obtained in the generation step, and then evaluate the MCAR (Multiple Completely at Random) condition, generate multiple imputations, and finally use bootstrap sampling for internal validation including optimism-corrected estimates of the performance parameters (AUC, accuracy, sensitivity, specificity, etc) including 95% Confidence Intervals.

In [121]:
import os
import pandas as pd
from scipy import stats

# pipelining with R
# https://blog.dominodatalab.com/lesser-known-ways-of-using-notebooks/

# Use the RWinOut instead of rpy2.ipython to get output on windows 
# https://bitbucket.org/rpy2/rpy2/issues/125/set_writeconsole-not-working-on-windows
# https://github.com/vitorcurtis/RWinOut
#%load_ext RWinOut
%load_ext rpy2.ipython

%R library(Hmisc)
%R library(mice)
%R library(boot)
%R library(rms)
%R library(ROCR)
%R library(ResourceSelection)
%R library(LogisticDx)

%matplotlib inline

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [122]:
# %load import_notebook.py
# Infraestructure to import a Jupyter notebook
# http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [123]:
from LogRegUtils import LogRegModel

## Initialization

In [124]:
# Data file and modeling prameters

src_data_file = '../data/data_normalized.csv'

risk_threshold = 0.5 # Classification threshold to define an outcome (1 if risk_threshold <= p, 0 otherwise)

num_imp = 100   # Number of multiple imputations
num_boot = 200  # Number of bootstrap samples

In [125]:
sel_model = 9
    
sl_only = False
lb_only = False

if sel_model == 1:
    model_name = 'all-minimal'
elif sel_model == 2:
    model_name = 'all-clinical-only'
elif sel_model == 3:
    model_name = 'all-parsimonious'
elif sel_model == 4:
    model_name = 'all-parsimonious-notemp'    
elif sel_model == 5:    
    model_name = 'sl-wellness-parsimonious'
    sl_only = True
elif sel_model == 6:    
    model_name = 'sl-wellness-parsimonious-notemp'
    sl_only = True    
elif sel_model == 7:
    model_name = 'sl-wellness-clinical-only'
    sl_only = True    
elif sel_model == 8:
    model_name = 'sl-wellness-minimal'
    sl_only = True
elif sel_model == 9:
    src_data_file = '../data/data.csv'
    model_name = 'sl-parsimonious'
    sl_only = True
    
elif sel_model == 10:    
    model_name = 'sl-wellness-parsimonious-noimp'
    sl_only = True
elif sel_model == 11:    
    model_name = 'sl-wellness-parsimonious-notemp-noimp'
    sl_only = True    
elif sel_model == 12:
    model_name = 'sl-wellness-clinical-only-noimp'
    sl_only = True    
elif sel_model == 13:
    model_name = 'sl-wellness-minimal-noimp'
    sl_only = True    

In [126]:
yvar = 'Disposition'

model_params = os.path.join(model_name, 'model.csv')
model = LogRegModel(model_params)

model_variables = [yvar] + model.getPredictors()

imp_variables = model_variables.copy()
if 'FeverTemperature' in model_variables:
    imp_variables += ['Fever']

model_formula = model.getGLMFormula(yvar)
model_imp_formula = model.getImputeFormula(yvar)

In [127]:
# Create model folder
from os import listdir, makedirs
from os.path import isfile, join, exists

model_folder = model_name
imp_folder = join(model_name, 'imp')
if not exists(imp_folder):
    makedirs(imp_folder)

# Load imputed data files, in case they exists. This allows to re-run bootstrap 
# calculations on same training set :-)    
imp_data_files = [join(imp_folder, f) for f in listdir(imp_folder) if isfile(join(imp_folder, f))]

print('Model variables:\n', model_variables, '\n')
print('Imputation variables:\n', imp_variables, '\n')
print('Model formula:\n', model_formula, '\n')
print('Imputation formula:\n', model_imp_formula, '\n')
print('Training files:', len(imp_data_files), '\n')

Model variables:
 ['Disposition', 'PatientAge', 'cycletime', 'malaria1', 'FeverTemperature', 'Jaundice', 'Bleeding', 'Breathlessness', 'SwallowingProblems', 'AstheniaWeakness', 'DaysSinceSymptomFeverOnset'] 

Imputation variables:
 ['Disposition', 'PatientAge', 'cycletime', 'malaria1', 'FeverTemperature', 'Jaundice', 'Bleeding', 'Breathlessness', 'SwallowingProblems', 'AstheniaWeakness', 'DaysSinceSymptomFeverOnset', 'Fever'] 

Model formula:
 Disposition~rcs(PatientAge,3,c(4.0,28.0,59.8))+cycletime+malaria1+rcs(FeverTemperature,3,c(36.3,37.3,39.2))+Jaundice+Bleeding+Breathlessness+SwallowingProblems+AstheniaWeakness+DaysSinceSymptomFeverOnset+cycletime * DaysSinceSymptomFeverOnset 

Imputation formula:
 ~Disposition+PatientAge+cycletime+malaria1+FeverTemperature+Jaundice+Bleeding+Breathlessness+SwallowingProblems+AstheniaWeakness+DaysSinceSymptomFeverOnset 

Training files: 0 



In [128]:
%%R 

# Random seed for reproducibility. Is reset at the beginning of each 
# calculating that involves randomization, so cells can be run in 
# arbitrary order and still reproduce results.

random_seed <- 151 

## Multiple imputation

In [129]:
%%R -i num_imp,model_imp_formula,imp_variables,src_data_file,model_folder,sl_only,lb_only -o imp_data_files

set.seed(random_seed)

src_data <- read.table(src_data_file, sep=",", header=TRUE, na.strings="\\N")
if (sl_only) {
    src_data <- src_data[src_data$ETUKey == 2 | src_data$ETUKey == 4 | src_data$ETUKey == 5, ]
}
if (lb_only) {
    src_data <- src_data[src_data$ETUKey == 1 | src_data$ETUKey == 3, ]
}
sat_impute <- aregImpute(as.formula(model_imp_formula), data=src_data, n.impute=num_imp)

imp_data_files <- character(0)
for (iter in 1:num_imp) {
  completed <- src_data
  imputed <- impute.transcan(sat_impute, imputation=iter, data=src_data, list.out=TRUE,
                             pr=FALSE, check=FALSE)
  completed[names(imputed)] <- imputed
  comp_data <- completed[unlist(imp_variables)]
  fn <- paste0(model_folder, '/imp', "/imputation-", iter, ".csv")
  write.csv(comp_data, file=fn, row.names=FALSE)
  imp_data_files <- c(imp_data_files, fn)    
}

Iteration 103 


## Internal validation by bootstrap sampling

In [130]:
%%R -i num_boot,risk_threshold,yvar,model_formula,model_folder,imp_data_files

set.seed(random_seed)

# (Adjusted) McFadden R2
# In the future we could use this library for calculation
# http://www.inside-r.org/packages/cran/bayloredpsych/docs/PseudoR2        
adjr2 <- function(obj) {
    # For the time being, just get numer of dofs in model (including intercept) 
    # using LogLik: http://stats.stackexchange.com/a/5580
    ll <- logLik(obj)
    K <- attr(ll, "df")
    r2 <- 1 - ((obj$deviance - K) / obj$null.deviance)
    return(r2)
}
        
calib <- function(probs,outcome,nbins=10) {
    c <- 0.0

    # Construct bins
    judgement_bins <- seq(0, nbins)/nbins

    # Which bin is each prediction in?
    bin_num <- .bincode(probs, judgement_bins, TRUE)

    for (j_bin in sort(unique(bin_num))) {
        # Is event in bin
        in_bin <- bin_num == j_bin
        
        # Predicted probability taken as average of preds in bin
        predicted_prob <- mean(probs[in_bin])
        
        # How often did events in this bin actually happen?
        true_bin_prob <- mean(outcome[in_bin])
        
        # Squared distance between predicted and true times num of obs
        c <- c + sum(in_bin) * (predicted_prob - true_bin_prob)^2
    } 
    cal <- c / length(probs)
    return(cal)
}
        
brier <- function(probs,outcome) {
    res <- mean((probs - outcome)^2)
    return(res)
}
 
accu <- function(probs,outcome) {
    preds = risk_threshold <= probs
    res <- 1 - mean(abs(preds - outcome))
    return(res)
}        

sens <- function(probs,outcome) {
    preds = risk_threshold <= probs
    tp <- sum(preds & outcome)
    fn <- sum(!preds & outcome)
#     print(sprintf("sens : %0.3f %0.3f %0.3f", tp, fn, tp / (tp + fn)))
    return(tp / (tp + fn))
}

spec <- function(probs,outcome) {
    preds = risk_threshold <= probs
    tn <- sum(!preds & !outcome)
    fp <- sum(preds & !outcome)
    return(tn / (tn + fp))
}

# Transform Z-scores back to score, and calculates CI at 95%
# https://stats.idre.ucla.edu/stata/faq/how-can-i-estimate-r-squared-for-a-model-estimated-with-multiply-imputed-data/
# Harel, O. (2009). The estimation of R2 and adjusted R2 in incomplete data sets using multiple imputation. Journal of Applied Statistics, 36(10), 1109-1118.
zinv <- function(values, N, M) {
    # Fist, we need the inter-imputation variance
    B <- sum((values - mean(values))^2) / (M - 1)

    # Now, we get the MI estimate of the variance of z
    V <- 1/(N-3) + B/(M+1)
     
    # The confidence interval, using the confidence level for 95%    
    Q <- mean(values)  
    ci_min <- tanh(Q - 1.959964*sqrt(V*Q))^2
    ci_max <- tanh(Q + 1.959964*sqrt(V*Q))^2
    val_mean <- tanh(Q)^2
    
    res <- c(val_mean, ci_min, ci_max)
    return(res)
}
     
optim <- function(src_dat, boot_idx) {
    src_idx <- 1:nrow(src_dat)
    boot_idx <- sample(src_idx, replace=TRUE)
    boot_dat <- src_dat[boot_idx,]

    boot_y <- as.matrix(boot_dat[,1])
    boot_x <- as.matrix(boot_dat[,2:ncol(boot_dat)])
  
    boot_mod <- glm(family="binomial", formula=mod_formula, data=boot_dat)

    # Get the indices of the rows not used in the bootstrap sample (the .632 method)
    rem_idx <- setdiff(src_idx, boot_idx)
    rem_dat <- train_data[rem_idx,] 
    rem_x <- as.matrix(rem_dat[,2:ncol(rem_dat)])
    rem_y <- as.matrix(rem_dat[,1])
    
    boot_prob <- predict(boot_mod, boot_dat, type="response")
    boot_pred <- prediction(boot_prob, boot_y)
    boot_auc <- performance(boot_pred, measure = "auc")

    rem_prob <- predict(boot_mod, rem_dat, type="response")
    rem_pred <- prediction(rem_prob, rem_y)
    auc <- performance(rem_pred, measure = "auc")
    
    # Values should not be 1, otherwise the Z-score transformation will give infinity
    rem_auc <- min(0.999, auc@y.values[[1]])    
    rem_bri <- min(0.999, brier(rem_prob, rem_y))
    rem_cal <- min(0.999, calib(rem_prob, rem_y))
    rem_acc <- min(0.999, accu(rem_prob, rem_y))
    rem_sens <- min(0.999, sens(rem_prob, rem_y))
    rem_spec <- min(0.999, spec(rem_prob, rem_y))    
    rem_r2 <- min(0.999, adjr2(boot_mod))

    # All values are returned as Z-scores using the method from 
    # http://www.ats.ucla.edu/stat/stata/faq/mi_r_squared.htm    
    auc_value <- atanh(sqrt(rem_auc))
    bri_value <- atanh(sqrt(rem_bri))
    cal_value <- atanh(sqrt(rem_cal))
    acc_value <- atanh(sqrt(rem_acc))
    sens_value <- atanh(sqrt(rem_sens))
    spec_value <- atanh(sqrt(rem_spec))    
    r2_value <- atanh(sqrt(rem_r2))
    
    res <- c(auc_value, bri_value, cal_value, acc_value, sens_value, spec_value, r2_value)
    return(res)     
}

auc_app_values <- vector(mode="numeric", length=length(imp_data_files))
auc_values <- vector(mode="numeric", length=length(imp_data_files))
bri_values <- vector(mode="numeric", length=length(imp_data_files)) 
cal_values <- vector(mode="numeric", length=length(imp_data_files))
acc_values <- vector(mode="numeric", length=length(imp_data_files))
sens_values <- vector(mode="numeric", length=length(imp_data_files))
spec_values <- vector(mode="numeric", length=length(imp_data_files))
r2_values <- vector(mode="numeric", length=length(imp_data_files))     

N <- 0
M <- length(imp_data_files) 
imp_iter <- 0
for (fn in imp_data_files) {
    imp_iter <- imp_iter + 1
    train_data <- read.table(fn, sep=",", header=TRUE)    
    N <- nrow(train_data)
    yvalues <- train_data[yvar]
    mod_formula <- as.formula(model_formula)
    model <- glm(family="binomial", formula=mod_formula, data=train_data)

    prob <- predict(model, train_data)
    pred <- prediction(prob, yvalues)
    auc <- performance(pred, measure = "auc")
    auc_app <- auc@y.values[[1]]
    
    bootres <- boot(train_data, optim, R=num_boot, parallel="multicore", ncpus=4)
        
    auc_app_values[imp_iter] <- atanh(sqrt(auc_app))
    auc_values[imp_iter] <- bootres$t[,1]
    bri_values[imp_iter] <- bootres$t[,2]
    cal_values[imp_iter] <- bootres$t[,3]
    acc_values[imp_iter] <- bootres$t[,4]
    sens_values[imp_iter] <- bootres$t[,5]
    spec_values[imp_iter] <- bootres$t[,6]
    r2_values[imp_iter] <- bootres$t[,7]
}
     
auc_app_mean <- zinv(auc_app_values, N, M)
auc_mean <- zinv(auc_values, N, M)
bri_mean <- zinv(bri_values, N, M)
cal_mean <- zinv(cal_values, N, M)
acc_mean <- zinv(acc_values, N, M)
sens_mean <- zinv(sens_values, N, M)
spec_mean <- zinv(spec_values, N, M)
r2_mean <- zinv(r2_values, N, M)

print(sprintf("Apparent AUC : %0.2f (%0.2f, %0.2f)", auc_app_mean[1], auc_app_mean[2], auc_app_mean[3]))
print(sprintf("Corrected AUC: %0.2f (%0.2f, %0.2f)", auc_mean[1], auc_mean[2], auc_mean[3]))
print(sprintf("Adjusted R2  : %0.2f (%0.2f, %0.2f)", r2_mean[1], r2_mean[2], r2_mean[3]))
print(sprintf("Brier score  : %0.2f (%0.2f, %0.2f)", bri_mean[1], bri_mean[2], bri_mean[3]))
print(sprintf("Calibration  : %0.2f (%0.2f, %0.2f)", cal_mean[1], cal_mean[2], cal_mean[3]))
print(sprintf("Accuracy     : %0.2f (%0.2f, %0.2f)", acc_mean[1], acc_mean[2], acc_mean[3]))   
print(sprintf("Sensitivity  : %0.2f (%0.2f, %0.2f)", sens_mean[1], sens_mean[2], sens_mean[3]))
print(sprintf("Specificity  : %0.2f (%0.2f, %0.2f)", spec_mean[1], spec_mean[2], spec_mean[3]))

sink(paste0(model_folder, "/boot.txt"), append=FALSE, split=FALSE)
print(sprintf("Apparent AUC : %0.2f (%0.2f, %0.2f)", auc_app_mean[1], auc_app_mean[2], auc_app_mean[3]))
print(sprintf("Corrected AUC: %0.2f (%0.2f, %0.2f)", auc_mean[1], auc_mean[2], auc_mean[3]))
print(sprintf("Adjusted R2  : %0.2f (%0.2f, %0.2f)", r2_mean[1], r2_mean[2], r2_mean[3]))
print(sprintf("Brier score  : %0.2f (%0.2f, %0.2f)", bri_mean[1], bri_mean[2], bri_mean[3]))
print(sprintf("Calibration  : %0.2f (%0.2f, %0.2f)", cal_mean[1], cal_mean[2], cal_mean[3]))
print(sprintf("Accuracy     : %0.2f (%0.2f, %0.2f)", acc_mean[1], acc_mean[2], acc_mean[3]))   
print(sprintf("Sensitivity  : %0.2f (%0.2f, %0.2f)", sens_mean[1], sens_mean[2], sens_mean[3]))
print(sprintf("Specificity  : %0.2f (%0.2f, %0.2f)", spec_mean[1], spec_mean[2], spec_mean[3]))
sink()

[1] "Apparent AUC : 0.83 (0.78, 0.87)"
[1] "Corrected AUC: 0.78 (0.72, 0.83)"
[1] "Adjusted R2  : 0.34 (0.27, 0.42)"
[1] "Brier score  : 0.19 (0.14, 0.25)"
[1] "Calibration  : 0.03 (0.01, 0.04)"
[1] "Accuracy     : 0.72 (0.65, 0.78)"
[1] "Sensitivity  : 0.80 (0.75, 0.85)"
[1] "Specificity  : 0.62 (0.54, 0.69)"
